# Training Models Baselines

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

## Meter 0

In [3]:
met0_tr = pd.read_pickle('kaggle/prepo_train_met0.pkl')
met0_vl = pd.read_pickle('kaggle/prepo_val_met0.pkl')

In [4]:
len(met0_tr), len(met0_vl)

(10661397, 1399513)

### Compute mean meter reading by building by month on training data

In [5]:
from tsm.data_utils import compute_and_add_mean_by_a_and_b, only_add_mean_by_a_and_b

In [6]:
met0_tr, tr_means_by_bid_and_m = compute_and_add_mean_by_a_and_b(met0_tr, mean_of='meter_reading', a='building_id', b='dt_m')

KeyboardInterrupt: 

In [45]:
met0_vl = only_add_mean_by_a_and_b(met0_vl, mean_of='meter_reading', a='building_id', b='dt_m', means_by_a_and_b=tr_means_by_bid_and_m)

#### Re-compress memory usage

In [7]:
from tsm.data_utils import compress_memory_usage

In [15]:
met0_tr = met0_tr.drop('timestamp', axis=1)

In [17]:
met0_tr, _ = compress_memory_usage(met0_tr)

In [18]:
met0_vl = met0_vl.drop('timestamp', axis=1)

In [19]:
met0_vl, _ = compress_memory_usage(met0_vl)

#### Save as pickle to save time later

In [20]:
met0_tr.to_pickle('kaggle/prepo_train_met0_bIDbyM_means.pkl')
met0_vl.to_pickle('kaggle/prepo_val_met0__bIDbyM_means.pkl')

In [5]:
import pickle

In [47]:
with open('kaggle/prepo_train_met0_tr_means_by_bid_and_m.pkl', 'wb') as f_out:
    pickle.dump(tr_means_by_bid_and_m, f_out)

#### Reloading: restart from here

In [3]:
met0_tr = pd.read_pickle('kaggle/prepo_train_met0_bIDbyM_means.pkl')
met0_vl = pd.read_pickle('kaggle/prepo_val_met0__bIDbyM_means.pkl')

In [6]:
with open('kaggle/prepo_train_met0_tr_means_by_bid_and_m.pkl', 'rb') as f_in:
    mt0_tr_means_by_bid_and_m = pickle.load(f_in)

## Baseline 1) LightGBM

In [9]:
import lightgbm as lgb

In [8]:
from tsm.eval_metrics import root_mean_squared_log_error

In [10]:
SEED = 0

In [11]:
lgb_reg_params = {'objective':'regression',  'boosting_type':'gbdt', 'metric':'rmse',
                  'n_jobs':-1, 'learning_rate':0.07, 'num_leaves': 2**8, 'max_depth':-1,
                  'tree_learner':'serial', 'colsample_bytree': 0.7, 'subsample_freq':1,
                  'subsample':0.5, 'max_bin':255, 'verbose':1, 'seed': SEED, 'early_stopping_rounds': 40}

In [11]:
xs = [x for x in list(met0_tr) if x not in ['timestamp', 'meter_reading']]
y = 'meter_reading'

In [12]:
met0_tr_x = met0_tr[xs]
met0_tr_y = met0_tr[y]
met0_vl_x = met0_vl[xs]
met0_vl_y = met0_vl[y]

In [62]:
%whos

Variable                          Type              Data/Info
-------------------------------------------------------------
SEED                              int               0
compute_and_add_mean_by_a_and_b   function          <function compute_and_add<...>_and_b at 0x7fb1a7f5bb70>
f_in                              BufferedReader    <_io.BufferedReader name=<...>_means_by_bid_and_m.pkl'>
f_out                             BufferedWriter    <_io.BufferedWriter name=<...>_means_by_bid_and_m.pkl'>
lgb                               module            <module 'lightgbm' from '<...>es/lightgbm/__init__.py'>
lgb_eval                          Dataset           <lightgbm.basic.Dataset object at 0x7fb16150ff60>
lgb_reg_params                    dict              n=15
lgb_train                         Dataset           <lightgbm.basic.Dataset object at 0x7fb16150f358>
met0_tr                           DataFrame                   building_id    <...>661397 rows x 59 columns]
met0_tr_x            

In [63]:
del met0_tr, met0_vl

In [13]:
mt0_lgb_train = lgb.Dataset(met0_tr_x, met0_tr_y)
mt0_lgb_eval = lgb.Dataset(met0_vl_x, met0_vl_y)
mt0_gbm_regress = lgb.train(lgb_reg_params, mt0_lgb_train, num_boost_round=2000, valid_sets=(mt0_lgb_train, mt0_lgb_eval),verbose_eval = 20)

/home/Ric/.local/lib/python3.5/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 119.759	valid_1's rmse: 124.142
[40]	training's rmse: 77.0525	valid_1's rmse: 85.5459
[60]	training's rmse: 69.6232	valid_1's rmse: 80.7385
[80]	training's rmse: 66.164	valid_1's rmse: 79.7223
[100]	training's rmse: 63.8183	valid_1's rmse: 78.8084
[120]	training's rmse: 61.7802	valid_1's rmse: 78.0157
[140]	training's rmse: 60.345	valid_1's rmse: 77.5614
[160]	training's rmse: 59.1499	valid_1's rmse: 77.4934
[180]	training's rmse: 58.2036	valid_1's rmse: 77.2936
[200]	training's rmse: 57.2973	valid_1's rmse: 77.3272
[220]	training's rmse: 56.3738	valid_1's rmse: 76.9794
[240]	training's rmse: 55.68	valid_1's rmse: 76.9387
[260]	training's rmse: 54.8786	valid_1's rmse: 76.9175
[280]	training's rmse: 54.2309	valid_1's rmse: 76.9433
Early stopping, best iteration is:
[252]	training's rmse: 55.2124	valid_1's rmse: 76.767


In [29]:
lgbm_tr_hat = (gbm_regress.predict(met0_tr_x, num_iteration=gbm_regress.best_iteration))
print('Training NRMLSE:', root_mean_squared_log_error(lgbm_tr_hat, met0_tr_y))

lgbm_vl_hat = (gbm_regress.predict(met0_vl_x, num_iteration=gbm_regress.best_iteration))
print('Validation NRMLSE:', root_mean_squared_log_error(lgbm_vl_hat, met0_vl_y))

/home/Ric/TSM/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Training NRMLSE: 0.6020488382795164
Validation NRMLSE: 0.6820828571711314


In [8]:
from sklearn.externals import joblib

/home/Ric/.local/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [15]:
joblib.dump(mt0_gbm_regress, 'models/mt0_gbm_regress.pkl')

['models/mt0_gbm_regress.pkl']

## Meter 1

In [16]:
met1_tr = pd.read_pickle('kaggle/prepo_train_met1.pkl')
met1_vl = pd.read_pickle('kaggle/prepo_val_met1.pkl')

In [17]:
met1_tr, mt_1tr_means_by_bid_and_m = compute_and_add_mean_by_a_and_b(met1_tr, mean_of='meter_reading', a='building_id', b='dt_m')

NameError: name 'compute_and_add_mean_by_a_and_b' is not defined

In [6]:
met1_vl = only_add_mean_by_a_and_b(met1_vl, mean_of='meter_reading', a='building_id', b='dt_m', means_by_a_and_b=mt_1tr_means_by_bid_and_m)

In [8]:
met1_tr = met1_tr.drop('timestamp', axis=1)

In [9]:
met1_tr, _ = compress_memory_usage(met1_tr)


Memory usage pre-compression was 1200.5119590759277
Memory usage after-compression was 677.9362182617188
This is  56.470592661446524% of the initial size


In [10]:
met1_vl = met1_vl.drop('timestamp', axis=1)

In [11]:
met1_vl, _ = compress_memory_usage(met1_vl)


Memory usage pre-compression was 155.6413917541504
Memory usage after-compression was 87.89166259765625
This is  56.470622375626824% of the initial size


In [12]:
met1_tr.to_pickle('kaggle/prepo_train_met1_bIDbyM_means.pkl')
met1_vl.to_pickle('kaggle/prepo_val_met1_bIDbyM_means.pkl')

In [14]:
with open('kaggle/prepo_train_met1_tr_means_by_bid_and_m.pkl', 'wb') as f_out:
    pickle.dump(mt_1tr_means_by_bid_and_m, f_out)

In [18]:
met1_tr = pd.read_pickle('kaggle/prepo_train_met1_bIDbyM_means.pkl')
met1_vl = pd.read_pickle ('kaggle/prepo_val_met1_bIDbyM_means.pkl')

In [19]:
xs = [x for x in list(met1_tr) if x not in ['timestamp', 'meter_reading']]
y = 'meter_reading'

In [20]:
met1_tr_x = met1_tr[xs]
met1_tr_y = met1_tr[y]
met1_vl_x = met1_vl[xs]
met1_vl_y = met1_vl[y]

In [21]:
mt1_lgb_train = lgb.Dataset(met1_tr_x, met1_tr_y)
mt1_lgb_eval = lgb.Dataset(met1_vl_x, met1_vl_y)
mt1_gbm_regress = lgb.train(lgb_reg_params, mt1_lgb_train, num_boost_round=2000, valid_sets=(mt1_lgb_train, mt1_lgb_eval),verbose_eval = 20)

/home/Ric/.local/lib/python3.5/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 2511.44	valid_1's rmse: 4669.04
[40]	training's rmse: 1472.94	valid_1's rmse: 3795.93
[60]	training's rmse: 1226.38	valid_1's rmse: 3689.2
[80]	training's rmse: 1090.02	valid_1's rmse: 3646.62
[100]	training's rmse: 1001.56	valid_1's rmse: 3636.51
[120]	training's rmse: 924.744	valid_1's rmse: 3633.53
[140]	training's rmse: 870.341	valid_1's rmse: 3650.83
Early stopping, best iteration is:
[105]	training's rmse: 980.043	valid_1's rmse: 3619.91


In [23]:
lgbm_tr_hat = (gbm_regress.predict(met1_tr_x, num_iteration=gbm_regress.best_iteration))
print('Training NRMLSE:', root_mean_squared_log_error(lgbm_tr_hat, met1_tr_y))

lgbm_vl_hat = (gbm_regress.predict(met1_vl_x, num_iteration=gbm_regress.best_iteration))
print('Validation NRMLSE:', root_mean_squared_log_error(lgbm_vl_hat, met1_vl_y))

/home/Ric/TSM/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Training NRMLSE: 1.5414553172411622
Validation NRMLSE: 1.7537242860741826


In [22]:
joblib.dump(mt1_gbm_regress, 'models/mt1_gbm_regress.pkl')

['models/mt1_gbm_regress.pkl']

In [34]:
%whos

Variable                      Type              Data/Info
---------------------------------------------------------
SEED                          int               0
f_in                          BufferedReader    <_io.BufferedReader name=<...>_means_by_bid_and_m.pkl'>
joblib                        module            <module 'sklearn.external<...>nals/joblib/__init__.py'>
lgb                           module            <module 'lightgbm' from '<...>es/lightgbm/__init__.py'>
lgb_reg_params                dict              n=15
pd                            module            <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
pickle                        module            <module 'pickle' from '/u<...>lib/python3.5/pickle.py'>
root_mean_squared_log_error   function          <function root_mean_squar<...>_error at 0x7fd0382beb70>


In [33]:
del mt0_tr_means_by_bid_and_m

In [26]:
del lgb_train, lgb_eval, lgbm_tr_hat, lgbm_vl_hat, met1_tr, met1_tr_x, met1_tr_y, met1_vl, met1_vl_x, met1_vl_y, mt_1tr_means_by_bid_and_m

In [28]:
del met1_tr, met1_vl

## Meter 2

In [46]:
met2_tr = pd.read_pickle('kaggle/prepo_train_met2.pkl')
met2_vl = pd.read_pickle('kaggle/prepo_val_met2.pkl')

In [47]:
met2_tr, mt_2tr_means_by_bid_and_m = compute_and_add_mean_by_a_and_b(met2_tr, mean_of='meter_reading', a='building_id', b='dt_m')

In [48]:
met2_vl = only_add_mean_by_a_and_b(met2_vl, mean_of='meter_reading', a='building_id', b='dt_m', means_by_a_and_b=mt_2tr_means_by_bid_and_m)

In [49]:
met2_tr = met2_tr.drop('timestamp', axis=1)

In [50]:
met2_tr, _ = compress_memory_usage(met2_tr)


Memory usage pre-compression was 774.2931060791016
Memory usage after-compression was 437.2479248046875
This is  56.47059509787478% of the initial size


In [51]:
met2_vl = met2_vl.drop('timestamp', axis=1)

In [52]:
met2_vl, _ = compress_memory_usage(met2_vl)


Memory usage pre-compression was 104.00534439086914
Memory usage after-compression was 58.73248291015625
This is  56.470639325446534% of the initial size


In [4]:
met2_tr = pd.read_pickle('kaggle/prepo_train_met2_bIDbyM_means.pkl')
met2_vl = pd.read_pickle('kaggle/prepo_val_met2_bIDbyM_means.pkl')

In [54]:
with open('kaggle/prepo_train_met2_tr_means_by_bid_and_m.pkl', 'wb') as f_out:
    pickle.dump(mt_2tr_means_by_bid_and_m, f_out)

In [5]:
xs = [x for x in list(met2_tr) if x not in ['timestamp', 'meter_reading']]
y = 'meter_reading'

In [6]:
met2_tr_x = met2_tr[xs]
met2_tr_y = met2_tr[y]
met2_vl_x = met2_vl[xs]
met2_vl_y = met2_vl[y]

In [12]:
mt2_lgb_train = lgb.Dataset(met2_tr_x, met2_tr_y)
mt2_lgb_eval = lgb.Dataset(met2_vl_x, met2_vl_y)
mt2_gbm_regress = lgb.train(lgb_reg_params, mt2_lgb_train, num_boost_round=2000, valid_sets=(mt2_lgb_train, mt2_lgb_eval),verbose_eval = 20)

/home/Ric/.local/lib/python3.5/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 125887	valid_1's rmse: 305748
[40]	training's rmse: 71291.8	valid_1's rmse: 277235
[60]	training's rmse: 56917.8	valid_1's rmse: 271659
[80]	training's rmse: 49772.7	valid_1's rmse: 268601
[100]	training's rmse: 44980.6	valid_1's rmse: 267856
[120]	training's rmse: 41184.2	valid_1's rmse: 268652
[140]	training's rmse: 38212.3	valid_1's rmse: 269253
Early stopping, best iteration is:
[102]	training's rmse: 44561.6	valid_1's rmse: 267463


In [60]:
lgbm_tr_hat = (gbm_regress.predict(met2_tr_x, num_iteration=gbm_regress.best_iteration))
print('Training NRMLSE:', root_mean_squared_log_error(lgbm_tr_hat, met2_tr_y))

lgbm_vl_hat = (gbm_regress.predict(met2_vl_x, num_iteration=gbm_regress.best_iteration))
print('Validation NRMLSE:', root_mean_squared_log_error(lgbm_vl_hat, met2_vl_y))

/home/Ric/TSM/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Training NRMLSE: 1.830378535525465
Validation NRMLSE: 1.9614942522066001


In [13]:
joblib.dump(mt2_gbm_regress, 'models/mt2_gbm_regress.pkl')

['models/mt2_gbm_regress.pkl']

In [62]:
%whos

Variable                          Type              Data/Info
-------------------------------------------------------------
SEED                              int               0
compress_memory_usage             function          <function compress_memory<...>_usage at 0x7f4a8449a1e0>
compute_and_add_mean_by_a_and_b   function          <function compute_and_add<...>_and_b at 0x7f4a66aab1e0>
f_out                             BufferedWriter    <_io.BufferedWriter name=<...>_means_by_bid_and_m.pkl'>
gbm_regress                       Booster           <lightgbm.basic.Booster object at 0x7f4a669e5ac8>
lgb                               module            <module 'lightgbm' from '<...>es/lightgbm/__init__.py'>
lgb_eval                          Dataset           <lightgbm.basic.Dataset object at 0x7f4a669e5f28>
lgb_reg_params                    dict              n=15
lgb_train                         Dataset           <lightgbm.basic.Dataset object at 0x7f4a669e5b38>
lgbm_tr_hat                

In [63]:
del xs, y, mt_2tr_means_by_bid_and_m, met1_v2, met2_tr, met2_tr_x, met2_tr_y, met2_vl, met2_vl_x, met2_vl_y, lgb_train, lgbm_tr_hat, lgb_eval, lgbm_vl_hat

## Meter 3

In [61]:
met3_tr = pd.read_pickle('kaggle/prepo_train_met3.pkl')
met3_vl = pd.read_pickle('kaggle/prepo_val_met3.pkl')

In [64]:
met3_tr, mt_3tr_means_by_bid_and_m = compute_and_add_mean_by_a_and_b(met3_tr, mean_of='meter_reading', a='building_id', b='dt_m')

In [65]:
met3_vl = only_add_mean_by_a_and_b(met3_vl, mean_of='meter_reading', a='building_id', b='dt_m', means_by_a_and_b=mt_3tr_means_by_bid_and_m)

In [66]:
met3_tr = met3_tr.drop('timestamp', axis=1)

In [67]:
met3_tr, _ = compress_memory_usage(met3_tr)


Memory usage pre-compression was 362.7468566894531
Memory usage after-compression was 204.8453369140625
This is  56.4706028836606% of the initial size


In [68]:
met3_vl = met3_vl.drop('timestamp', axis=1)

In [69]:
met3_vl, _ = compress_memory_usage(met3_vl)


Memory usage pre-compression was 47.116458892822266
Memory usage after-compression was 26.60699462890625
This is  56.47070101220949% of the initial size


In [14]:
met3_tr = pd.read_pickle('kaggle/prepo_train_met3_bIDbyM_means.pkl')
met3_vl = pd.read_pickle('kaggle/prepo_val_met3_bIDbyM_means.pkl')

In [71]:
with open('kaggle/prepo_train_met3_tr_means_by_bid_and_m.pkl', 'wb') as f_out:
    pickle.dump(mt_3tr_means_by_bid_and_m, f_out)

In [15]:
xs = [x for x in list(met3_tr) if x not in ['timestamp', 'meter_reading']]
y = 'meter_reading'

In [16]:
met3_tr_x = met3_tr[xs]
met3_tr_y = met3_tr[y]
met3_vl_x = met3_vl[xs]
met3_vl_y = met3_vl[y]

In [17]:
mt3_lgb_train = lgb.Dataset(met3_tr_x, met3_tr_y)
mt3_lgb_eval = lgb.Dataset(met3_vl_x, met3_vl_y)
mt3_gbm_regress = lgb.train(lgb_reg_params, mt3_lgb_train, num_boost_round=2000, valid_sets=(mt3_lgb_train, mt3_lgb_eval),verbose_eval = 20)

/home/Ric/.local/lib/python3.5/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 40 rounds
[20]	training's rmse: 1355.6	valid_1's rmse: 1789.37
[40]	training's rmse: 1089.65	valid_1's rmse: 1677.05
[60]	training's rmse: 954.936	valid_1's rmse: 1658.86
[80]	training's rmse: 858.759	valid_1's rmse: 1665.79
[100]	training's rmse: 786.847	valid_1's rmse: 1667.93
Early stopping, best iteration is:
[64]	training's rmse: 933.405	valid_1's rmse: 1654.98


In [75]:
lgbm_tr_hat = (gbm_regress.predict(met3_tr_x, num_iteration=gbm_regress.best_iteration))
print('Training NRMLSE:', root_mean_squared_log_error(lgbm_tr_hat, met3_tr_y))

lgbm_vl_hat = (gbm_regress.predict(met3_vl_x, num_iteration=gbm_regress.best_iteration))
print('Validation NRMLSE:', root_mean_squared_log_error(lgbm_vl_hat, met3_vl_y))

/home/Ric/TSM/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Training NRMLSE: 1.9423154769341195
Validation NRMLSE: 2.1001962268893863


In [18]:
joblib.dump(mt3_gbm_regress, 'models/mt3_gbm_regress.pkl')

['models/mt3_gbm_regress.pkl']

## Computing overall baselines

In [78]:
len_T0 = 10661397
len_V0 = 1399513
acc_T0 = 0.6020488382795164
acc_V0 = 0.6820828571711314
len_T1 = 3702435
len_V1 = 480005
acc_T1 = 1.5414553172411622
acc_V1 = 1.7537242860741826
len_T2 = 2387956
len_V2 = 320757
acc_T2 = 1.830378535525465
acc_V2 = 1.9614942522066001
len_T3 = 1118728
len_V3 = 145309
acc_T3 = 1.9423154769341195
acc_V3 = 2.1001962268893863

In [84]:
len_T = len_T0 + len_T1 + len_T2 + len_T3
len_V = len_V0 + len_V1 + len_V2 + len_V3

In [93]:
acc_T = acc_T0 * (len_T0 / len_T) + acc_T1 * (len_T1 / len_T) + acc_T2 * (len_T2 / len_T) + acc_T3 * (len_T3 / len_T)

In [95]:
acc_V = acc_V0 * (len_V0 / len_V) + acc_V1 * (len_V1 / len_V) + acc_V2 * (len_V2 / len_V) + acc_V3 * (len_V3 / len_V)

In [96]:
acc_T, acc_V

(1.0447155477119576, 1.1641964973385228)

In [97]:
L = len_T + len_V

In [98]:
acc_T * len_T / L + acc_V * len_V / L

1.0585783898896715